Task 2 prepara data

In [ ]:
!pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATASET_PATH = "data\deforestation-in-ukraine\S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510"

In [3]:
# data_preparation.ipynb
import os
import glob
import cv2
import numpy as np
import rasterio
from rasterio.plot import reshape_as_image

def extract_and_preprocess_images(dataset_path, max_dimension=2048):
    """
    Extract and preprocess images for training.

    Args:
        dataset_path (str): Path to the dataset.
        max_dimension (int): Maximum dimension for resizing images.

    Returns:
        images (list): Preprocessed images as numpy arrays.
    """
    pattern = os.path.join(dataset_path, '**', 'IMG_DATA', '*_TCI.jp2')
    image_paths = glob.glob(pattern, recursive=True)

    images = []
    for image_path in image_paths:
        # Load image using rasterio
        with rasterio.open(image_path, "r", driver="JP2OpenJpeg") as src:
            image = src.read()
            image = reshape_as_image(image)

        # Resize and normalize the image
        height, width, _ = image.shape
        scale = max_dimension / max(height, width)
        new_size = (int(width * scale), int(height * scale))
        image_resized = cv2.resize(image, new_size, interpolation=cv2.INTER_CUBIC)
        image_normalized = cv2.normalize(image_resized, None, alpha=0, beta=1,
                                         norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        images.append(image_normalized)

    return np.array(images)

def save_dataset(images, save_path):
    """
    Save the dataset as a NumPy file for training.

    Args:
        images (numpy array): Array of images.
        save_path (str): Path to save the dataset.
    """
    np.save(save_path, images)

# Prepare and save the dataset
images = extract_and_preprocess_images(DATASET_PATH)
save_dataset(images, "prepared_dataset.npy")
print("Data for training is created")

Data for training is created
